### 0. Prereqs

- A W&B account (https://wandb.ai/signup). 
- python>=3.9
- wandb installed and logged in.

In [ ]:
# I'm installing the core wand and nbformat library.
# nbformat is required to save notebook history.
# scikit-learn and joblib are not necessary but are used for this demo.
%pip install python-dotenv wandb nbformat scikit-learn joblib -q

I created a .env file with the entries. You need to modify and use values specific to your registry.

```text
WANDB_API_KEY=xxxxxxxxxxxxxxxxxxxxxxxxxxx
WANDB_ORG_ENTITY=olaekdahl-ciracon-org
WANDB_TEAM_ENTITY=olaekdahl-ciracon
WANDB_PROJECT=demo
WANDB_COLLECTION=iris-clf
```

In [1]:
# You will be asked for an API key. 
# You'll find that after you create your W&B account.
# I store my API key in a .env file.
import os
from dotenv import load_dotenv
load_dotenv()
import wandb
wandb.login(key=os.getenv("WANDB_API_KEY"))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ola/.netrc
wandb: Currently logged in as: olaekdahl to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

### 1. What the W&B Registry is (mental model)

- Registry = your organization's central catalog of versioned artifacts (datasets, models, etc.). It's built on Artifacts and organizes versions into collections. Think: "Registered Model" or "Registered Dataset" with version history, lineage, and governance. 

- W&B ships core registries for Models and Datasets (you can also create custom registries).

## 2. Registry UI

- Go to Registry in the W&B app (https://wandb.ai/registry/). You'll see core registries (Models, Datasets). You can also create custom ones. 

- (Optional) Create protected aliases like production/staging in the registry settings. These are locked labels you’ll apply to versions to prevent accidental changes. 

- (Optional) Create a collection (e.g., iris-classifier) under the Models registry. You can also create collections programmatically when you first link a version. 

## 3. Train → log artifact → link to Registry

In [3]:
import os, time
import joblib
import wandb
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# IMPORTANT: use your ORG's team entity (not your personal user) so you can link to the Registry.
WANDB_TEAM_ENTITY = os.getenv("WANDB_TEAM_ENTITY", "olaekdahl-ciracon")
PROJECT = os.getenv("WANDB_PROJECT", "demo")
REGISTRY_NAME = os.getenv("WANDB_REGISTRY", "Model")  # core "Models" registry
COLLECTION_NAME = os.getenv("WANDB_COLLECTION", "iris-clf")  # will be created if absent

run = wandb.init(entity=WANDB_TEAM_ENTITY, project=PROJECT, job_type="train")

# 1) Train a trivial model
X, y = load_iris(return_X_y=True)
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)
clf = LogisticRegression(max_iter=1000, random_state=42).fit(Xtr, ytr)
acc = clf.score(Xte, yte)
wandb.log({"test_acc": acc})

# 2) Save model locally and create an Artifact
os.makedirs("artifacts", exist_ok=True)
model_path = "artifacts/iris_logreg.joblib"
joblib.dump(clf, model_path)

artifact = wandb.Artifact(
    name="iris_logreg",          # logical artifact name within the project
    type="model",                # collection will accept "model" type
    metadata={"test_acc": acc},  # optional metadata
)

artifact.add_file(model_path)

# 3) Link this artifact version into the org-wide Registry collection
target_path = f"wandb-registry-{REGISTRY_NAME}/{COLLECTION_NAME}"

# Add non-protected aliases programmatically (e.g., "candidate-YYYYMMDD")
date_alias = time.strftime("candidate-%Y%m%d")
run.link_artifact(
    artifact=artifact,
    target_path=target_path,
    aliases=[date_alias, "latest-candidate"]
)

print(f"Linked to Registry collection: {target_path} with aliases: {date_alias}, latest-candidate")
run.finish()


Linked to Registry collection: wandb-registry-Model/iris-clf with aliases: candidate-20250809, latest-candidate


test_acc,▁
test_acc,1


### Notes:

- `run.link_artifact()` both logs the artifact and links that version into the registry's collection. If the collection doesn't exist, it's created. 

- You can add aliases at link time. Use protected aliases (e.g., production) for governance. Create them in the registry UI, then you can add them programmatically. 

- Artifacts must be logged under a team entity to be linkable.

## 4. Promote a candidate to production

- Create the protected alias production in the registry settings:
    1. Navigate to the **W&B Registry**.
    2. Click the **View details** button in a collection.
    3. Within the **Versions** section, click the View button for a specific artifact version.
    4. Click the + button to add one or more aliases next to the Aliases field. 

<img src="images/wandb-registry-details.png" alt="W&B Registry Details" width="800"/>

## 5. Consume a model from the Registry by alias

In [10]:
import os, joblib, wandb
from dotenv import load_dotenv
load_dotenv()

WANDB_ORG_ENTITY = os.getenv("WANDB_ORG_ENTITY", "olaekdahl-ciracon-org")
WANDB_TEAM_ENTITY = os.getenv("WANDB_TEAM_ENTITY", "olaekdahl-ciracon")  # must match the org/team that owns the registry
PROJECT = os.getenv("WANDB_PROJECT", "demo")

REGISTRY = "model"
COLLECTION = os.getenv("WANDB_COLLECTION", "iris-clf")
ALIAS = os.getenv("MODEL_ALIAS", "production")

run = wandb.init(entity=WANDB_TEAM_ENTITY, project=PROJECT, job_type="inference")

# olaekdahl-ciracon-org/wandb-registry-model/iris-clf:production
artifact_name = f"{WANDB_ORG_ENTITY}/wandb-registry-{REGISTRY}/{COLLECTION}:{ALIAS}"
print("Fetching:", artifact_name)

art = run.use_artifact(artifact_or_name = artifact_name)
model_dir = art.download()
clf = joblib.load(os.path.join(model_dir, "iris_logreg.joblib"))

print(f"Loaded model {clf} for alias: {ALIAS}")


Fetching: olaekdahl-ciracon-org/wandb-registry-model/iris-clf:production


wandb:   1 of 1 files downloaded.  


Loaded model LogisticRegression(max_iter=1000, random_state=42) for alias: production
